## Week 04 : 웹 크롤링

In [2]:
import re
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime, timedelta

import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from collections import Counter
from itertools import chain
from operator import itemgetter
        
basic_header = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5)\AppleWebKit 537.36 (KHTML, like Gecko) Chrome",
                "Accept":"text/html,application/xhtml+xml,application/xml;\q=0.9,imgwebp,*/*;q=0.8"}

In [31]:
x = ['a', 'b', 'c']
y = [] # ['am', 'bm', 'cm']

for i in x :
    y.append(i+'m')

y

['am', 'bm', 'cm']

In [3]:
url = 'http://underscore.kr/'

session = requests.Session()
req = session.get(url, headers=basic_header, allow_redirects=True)
soup = BeautifulSoup(req.text, 'lxml')

In [165]:
for each_time in soup.find_all('time') :
    print(each_time['class'])

['entry-date', 'published', 'updated']
['entry-date', 'published', 'updated']
['entry-date', 'published']
['updated']
['entry-date', 'published']
['updated']
['entry-date', 'published', 'updated']
['entry-date', 'published']
['updated']


In [18]:
soup.find('h2')

<h2 class="title-post entry-title"><a href="http://underscore.kr/istrollingcontagious/" rel="bookmark">왜 자꾸 어그로를 끄는 거야!</a></h2>

In [22]:
soup.find('h2').a

<a href="http://underscore.kr/istrollingcontagious/" rel="bookmark">왜 자꾸 어그로를 끄는 거야!</a>

In [24]:
soup.find('h2').a['href']

'http://underscore.kr/istrollingcontagious/'

In [34]:
soup.find_all('h2')

[<h2 class="title-post entry-title"><a href="http://underscore.kr/istrollingcontagious/" rel="bookmark">왜 자꾸 어그로를 끄는 거야!</a></h2>,
 <h2 class="title-post entry-title"><a href="http://underscore.kr/politaku/" rel="bookmark">페이스북으로 ‘정잘알’이 될 수 있을까?</a></h2>,
 <h2 class="title-post entry-title"><a href="http://underscore.kr/genderindanger/" rel="bookmark">당신은 남자답지 않습니다</a></h2>,
 <h2 class="title-post entry-title"><a href="http://underscore.kr/goodhigschoolanduniv/" rel="bookmark">좋은 고등학교를 가면 대학 입시에서 유리할까?</a></h2>,
 <h2 class="title-post entry-title"><a href="http://underscore.kr/hotweatherkillspeople/" rel="bookmark">덥다, 더워! 데이터가 알려주는 더위와 자살률의 관계</a></h2>,
 <h2 class="title-post entry-title"><a href="http://underscore.kr/arenoobsmoreconfident/" rel="bookmark">진짜로 잘 모르는 사람이 더 용감할까?</a></h2>]

In [50]:
title_list = []
suburl_list = []
for each in soup.find_all('h2') :
    title_list.append(each.a.text)
    suburl_list.append(each.a['href'])

In [56]:
title_list = [each.a.text for each in soup.find_all('h2')]
suburl_list = [each.a['href'] for each in soup.find_all('h2')]

In [115]:
underscoredf = pd.DataFrame(zip(title_list, suburl_list), columns=['title', 'url'])

In [116]:
len_url_list = []
for each in underscoredf['url'] :
    len_url_list.append(len(each))

len_title_list = []
for each in underscoredf['title'] :
    len_title_list.append(len(each))

In [117]:
underscoredf['len_title'] = len_title_list
underscoredf['len_url'] = len_url_list

In [118]:
underscoredf[['title', 'len_url', 'url']]

,title,len_url,url
0,왜 자꾸 어그로를 끄는 거야!,42,http://underscore.kr/istrollingcontagious/
1,페이스북으로 ‘정잘알’이 될 수 있을까?,30,http://underscore.kr/politaku/
2,당신은 남자답지 않습니다,36,http://underscore.kr/genderindanger/
3,좋은 고등학교를 가면 대학 입시에서 유리할까?,42,http://underscore.kr/goodhigschoolanduniv/
4,"덥다, 더워! 데이터가 알려주는 더위와 자살률의 관계",43,http://underscore.kr/hotweatherkillspeople/
5,진짜로 잘 모르는 사람이 더 용감할까?,43,http://underscore.kr/arenoobsmoreconfident/


In [119]:
underscoredf['source'] = 'underscore'

In [120]:
underscoredf.columns

Index(['title', 'url', 'len_title', 'len_url', 'source'], dtype='object')

In [144]:
underscoredf['len_title']

0    16
1    22
2    13
3    25
4    29
5    21
Name: len_title, dtype: int64

In [151]:
underscoredf['digit_len_title'] = underscoredf['len_title'].astype(str).str.len()

In [155]:
underscoredf['doubled_len_title'] = underscoredf['len_title']*2